In [1]:
import pandas as pd
import json
import os
from openai import OpenAI #estamos la clase concreta OpenAI del módulo openai
from dotenv import load_dotenv #importamos una función concreta del módulo
load_dotenv("template.env")

True

In [2]:
# Acceder a la clave de API de OpenAI
api_key = os.getenv("OPENAI_API_KEY")

# Asegurarte de que la clave de API se haya cargado correctamente
if api_key is None:
    raise ValueError("La clave de API no está configurada en el archivo .env")
    
client = OpenAI() #creando un objeto de la clase

In [3]:
dataset_folder = os.getenv("DATASET_FOLDER")
dataset_path = str(dataset_folder) + "MMLU_completo.xlsx"

df = pd.read_excel(dataset_path)
#df = df.sample(20)
df.head()

,instruction,option_a,option_b,option_c,option_d,answer,id
0,For which of these two scenarios does the main...,"Wrong, Wrong","Wrong, Not wrong","Not wrong, Wrong","Not wrong, Not wrong",D,moral_scenarios_test
1,For which of these two scenarios does the main...,"Wrong, Wrong","Wrong, Not wrong","Not wrong, Wrong","Not wrong, Not wrong",A,moral_scenarios_test
2,For which of these two scenarios does the main...,"Wrong, Wrong","Wrong, Not wrong","Not wrong, Wrong","Not wrong, Not wrong",D,moral_scenarios_test
3,For which of these two scenarios does the main...,"Wrong, Wrong","Wrong, Not wrong","Not wrong, Wrong","Not wrong, Not wrong",B,moral_scenarios_test
4,For which of these two scenarios does the main...,"Wrong, Wrong","Wrong, Not wrong","Not wrong, Wrong","Not wrong, Not wrong",D,moral_scenarios_test


In [14]:
#FUNCTION DECLARATION

#MODEL = "gpt-4o-mini"
#MODEL = "gpt-4o"
MODEL = "gpt-4.1-nano"

#Generate description of multiple question
def generate_description(instr,op_a,op_b,op_c,op_d):
	description = json.dumps({
        "instruction": instr,
        "options": {
            "A": op_a,
            "B": op_b,
            "C": op_c,
            "D": op_d
        }
    })
	return description

#Generate one task
def generate_task(index,prompt,desc):
	task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # Esto es lo que tendrías en tu llamada a la API de Chat Completions
            "model": MODEL,
            "temperature": 0,
            "response_format": { 
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system",
                    "content": prompt
                },
                {
                    "role": "user",
                    "content": desc
                }
            ],
        }
    }
	return task

def extract_data_paraphrase(new_line):
	res = new_line["response"]["body"]["choices"][0]["message"]["content"]
	res = json.loads(res)
	return res

def get_keys(json_object):
	res = []
	for i in json_object:
		res.append(i)
	return res

def create_task_from_json(json_object,index,prompt,rename_func=None):
	if (rename_func == None):
		rename_func = ['instruction','option_a','option_b','option_c','option_d']
	try:
		instruction = json_object[rename_func[0]]
		option_a = json_object[rename_func[1]]
		option_b = json_object[rename_func[2]]
		option_c = json_object[rename_func[3]]
		option_d = json_object[rename_func[4]]
	except:
		print(json_object)
		print(index)

	description = generate_description(
		instruction, option_a, option_b, option_c, option_d,
	)
    
	task = generate_task(
		index, prompt,description,
	)
	return task

def create_task_array_from_dataframe(df,prompt):
	tasks = []
	for index, row in df.iterrows():
		task = create_task_from_json(row,index,prompt)
		tasks.append(task)
	return tasks


def create_file_from_tasks(tasks,file_name):
	with open(file_name, 'w') as file:
		for obj in tasks:
			file.write(json.dumps(obj) + '\n')


def create_batch(file_name):
	batch_file = client.files.create(
		file = open(file_name, "rb"),
		purpose = "batch"
	)
	batch_job = client.batches.create(
		input_file_id = batch_file.id,
		endpoint = "/v1/chat/completions",
		completion_window = "24h"
	)
	return batch_job

def get_line_file(file_name,line,extract_func):
	with open(file_name, 'r') as f:
		for line_number, theline in enumerate(f):
			if line_number == line:
				res = theline
				break
	res = json.loads(res)
	return extract_func(res)


def extract_none(line):return line


def create_task_array_from_filename(file_name,prompt,extract_func,rename_func,error_file):
	tasks = []
	num_errors = 0
	with open(file_name, 'r') as f:
		lines = len(f.readlines())

	for i in range(0,lines):
		try:
			line_new = get_line_file(file_name,i,extract_func)
			task = create_task_from_json(line_new,i,prompt,rename_func)
			tasks.append(task)
		except:
			line_new = get_line_file(file_name.replace("_result",""),i,extract_none)
			with open(error_file, 'a') as f:
				f.write(json.dumps(line_new) + '\n')
			num_errors+=1
			#print(f"error in line[{i}]:\n{line_new}")
	print(f"number of errors = {num_errors}")
	return tasks


In [10]:
#PROMPTS

#PARAPHRASING PROMPT
categorize_system_prompt_paraphrase ='''
Your goal is to paraphrase multiple choice questions from JSON objects. You will be provided with a JSON object containing a multiple choice question and you will output a JSON object with the reworded question.

For each row paraphrase the question while maintaining the original meaning and paraphrase the options as well. Ensure the paraphrase is at least 500 characters long between options and question.

The output JSON objects should be in the following format:

{paraphrased_question: string, option_a: string, option_b: string, option_c: string, option_d: string,}

Ensure that the question is presented differently but conveys the same idea. Keep the JSON format in the output with '{' and '}'.
'''

#ANSWER PROMPT
categorize_system_prompt_answer = '''
Your goal is to evaluate multiple choice questions from JSON objects. You will be provided with a JSON object containing a multiple choice question and you will output a JSON object with the evaluation of the question.

For each row evaluate the question.

The output JSON objects should be in the following format:

{answer: string, // The selected option key for the question, limited to 'A', 'B', 'C', or 'D'}

Keep the JSON format in the answer with '{' and '}'.
'''

In [8]:
#Set output folder
output_folder = os.getenv("OUTPUT_FOLDER")
def out_file(file_name): return (str(output_folder) + file_name)

In [11]:
#PARAPHRASING TASK
tasks_array = [create_task_array_from_dataframe(df,categorize_system_prompt_paraphrase)]
file_array = [out_file("batch_job_mmlu_nano_paraphrase.jsonl")]

In [13]:
file_name = out_file("batch_job_mmlu_nano_paraphrase_result.jsonl")
rename_func = ['paraphrased_question','option_a','option_b','option_c','option_d']

error_file_name = out_file("batch_job_mmlu_answer_paraphrase_error.jsonl")

paraphrased_task = create_task_array_from_filename(file_name,categorize_system_prompt_answer,extract_data_paraphrase,rename_func,error_file_name)

{'paraphrased_question': "Create a comprehensive truth table to analyze the logical structure of the given argument. After constructing the table, assess whether the argument holds as valid or invalid based on the truth values. If the argument is found to be invalid, select the option that provides a specific counterexample demonstrating its invalidity. Keep in mind that multiple counterexamples might exist, so consider different combinations of truth values for the involved propositions.\n\nThe argument involves the following premises: 'If P then Q' and 'Q and R', leading to the conclusion '~P and R'. Using the truth table, determine the validity of this argument and identify any counterexamples if it is invalid.\n\nOptions:\nA. The argument is valid.\nB. The argument is invalid; a counterexample occurs when P, R, and Q are all true.\nC. The argument is invalid; a counterexample occurs when P and Q are true, but R is false.\nD. The argument is invalid; a counterexample occurs when R a

In [15]:
ba_jo_error= create_batch(error_file_name)

In [18]:
batch = ba_jo_error
batch = client.batches.retrieve(batch.id)
print(batch)
result_file_id = batch.output_file_id
print(batch.status)

Batch(id='batch_680289ac091c819098f69d4671e12c23', completion_window='24h', created_at=1744996780, endpoint='/v1/chat/completions', input_file_id='file-3KY3E5G1FXXNUctzF9EBN9', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1744996989, error_file_id=None, errors=None, expired_at=None, expires_at=1745083180, failed_at=None, finalizing_at=1744996986, in_progress_at=1744996781, metadata=None, output_file_id='file-LZ3PtZVbN9aUkZ23RZWL65', request_counts=BatchRequestCounts(completed=22, failed=0, total=22))
completed


In [19]:
batch = ba_jo_error
batch = client.batches.retrieve(batch.id)
result_file_id = batch.output_file_id

result = client.files.content(result_file_id).content

result_file_name = error_file_name.replace(".json","_result.json")

with open(result_file_name, 'wb') as file:
	file.write(result)

In [ ]:
#ANSWERING TASK
tasks_array = [
	create_task_array_from_dataframe(df,categorize_system_prompt_answer),
	paraphrased_task
	]
file_array = [out_file("batch_job_mmlu_nano_answer_original.jsonl"),out_file("batch_job_mmlu_nano_answer_paraphrase.jsonl")]

error in line[3973]:
{'custom_id': 'task-3973', 'method': 'POST', 'url': '/v1/chat/completions', 'body': {'model': 'gpt-4.1-nano', 'temperature': 0, 'response_format': {'type': 'json_object'}, 'messages': [{'role': 'system', 'content': "\nYour goal is to paraphrase multiple choice questions from JSON objects. You will be provided with a JSON object containing a multiple choice question and you will output a JSON object with the reworded question.\n\nFor each row paraphrase the question while maintaining the original meaning and paraphrase the options as well. Ensure the paraphrase is at least 500 characters long between options and question.\n\nThe output JSON objects should be in the following format:\n\n{paraphrased_question: string, option_a: string, option_b: string, option_c: string, option_d: string,}\n\nEnsure that the question is presented differently but conveys the same idea. Keep the JSON format in the output with '{' and '}'.\n"}, {'role': 'user', 'content': '{"instruction"

In [12]:
#GENERATE TASK FILES
for i in range(0,len(tasks_array)):
	create_file_from_tasks(tasks_array[i],file_array[i])

In [13]:
#GENERATE BATCH
batch_jobs = []
for i in range(0,len(tasks_array)):
	ba_jo= create_batch(file_array[i])
	batch_jobs.append(ba_jo)

In [22]:
#COMPLETION_CHECK
for i in range(0,len(batch_jobs)):
	batch = batch_jobs[i]
	batch = client.batches.retrieve(batch.id)
	print(batch)
	result_file_id = batch.output_file_id
	print(batch.status)

Batch(id='batch_6802635350a48190bd2634640d94d4f4', completion_window='24h', created_at=1744986963, endpoint='/v1/chat/completions', input_file_id='file-1jTwssaSzY8DqVJsfU8r7B', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1744990785, error_file_id=None, errors=None, expired_at=None, expires_at=1745073363, failed_at=None, finalizing_at=1744989168, in_progress_at=1744986968, metadata=None, output_file_id='file-KZPCP2hRNMR2U6G54Aak8s', request_counts=BatchRequestCounts(completed=14042, failed=0, total=14042))
completed


In [23]:
#OUTPUT FILES GENERATOR
for i in range(0,len(batch_jobs)):
	batch = batch_jobs[i]
	batch = client.batches.retrieve(batch.id)
	result_file_id = batch.output_file_id

	result = client.files.content(result_file_id).content

	result_file_name = file_array[i].replace(".json","_result.json")

	with open(result_file_name, 'wb') as file:
		file.write(result)

In [24]:
import re

def clean_text(text):
    if isinstance(text, str):
        # Eliminar caracteres de control ASCII (excepto saltos de línea/tabulación)
        text = re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]", "", text)
        return text.strip()
    return text

#Cleaning function
def clean():
	with open(out_file("batch_job_mmlu_nano_paraphrase.jsonl"), 'r') as f:
		lines = len(f.readlines())
	
	rows = []
	stats = [0,0]

	f_ph = out_file("batch_job_mmlu_nano_paraphrase_result.jsonl")
	f_a = out_file("batch_job_mmlu_nano_answer_original_result.jsonl")
	f_ph_a = out_file("batch_job_mmlu_nano_answer_paraphrase_result.jsonl")

	for i in range(0,lines):
	
		dt_ph = get_line_file(f_ph,i,extract_data_paraphrase)
		dt_a = get_line_file(f_a,i,extract_data_paraphrase)
		dt_ph_a = get_line_file(f_ph_a,i,extract_data_paraphrase)

		row = df.iloc[i]
		rows.append([
       	    row['instruction'],
         	row['option_a'],
         	row['option_b'],
         	row['option_c'],
         	row['option_d'],
         	row['answer'],
			dt_a["answer"],
			clean_text(dt_ph["paraphrased_question"]),
			clean_text(dt_ph["option_a"]),
			clean_text(dt_ph["option_b"]),
			clean_text(dt_ph["option_c"]),
			clean_text(dt_ph["option_d"]),
			dt_ph_a["answer"],
		])

		if(row['answer'] == dt_a["answer"]): stats[0]+=1
		if(row['answer'] == dt_ph_a["answer"]): stats[1]+=1
	
	
	stats[0] = stats[0]/lines
	stats[1] = stats[1]/lines
	
	stats = pd.DataFrame([{'acuracy': stats[0],'acuracy_ph': stats[1],}])
	return pd.DataFrame(rows),stats

file_name = out_file('FinalResults_gpt41nano.xlsx')
clean_dtset,stat = clean()


with pd.ExcelWriter(file_name) as writer:
	clean_dtset.to_excel(writer, sheet_name='Results')
	stat.to_excel(writer, sheet_name='Stats',engine='xlsxwriter')

FileNotFoundError: [Errno 2] No such file or directory: 'output_files/batch_job_mmlu_nano_answer_original_result.jsonl'

new_row = {
       	    'question': row['instruction'],
         	'A': row['option_a'],
         	'B': row['option_b'],
         	'C': row['option_c'],
         	'D': row['option_d'],
         	'correct_answer': row['answer'],
			'llm_answer': dt_a["answer"],
			'question_prphr' : dt_ph["paraphrased_question"],
			'A_prphr' : dt_ph["option_a"],
			'B_prphr' : dt_ph["option_b"],
			'C_prphr' : dt_ph["option_c"],
			'D_prphr' : dt_ph["option_d"],
			'llm_answer_prphr': dt_ph_a["answer"],
        }